In [ ]:
import numpy as np

In [ ]:
def ArmijoLineSearch(x, f, d, g, c1, problem, method, options):
    """
    Perform an Armijo line search to find a step size that satisfies the Armijo condition.

    Args:
        x (numpy.ndarray): Current iterate.
        f (float): Current objective function value.
        d (numpy.ndarray): Search direction.
        g (numpy.ndarray): Gradient of the objective function.
        c1 (float): Armijo condition parameter.
        problem (Problem): Problem object defining the optimization problem.
        method (Method): Method object defining optimization method settings.
        options (Options): Options object defining optimization options.

    Returns:
        float: The step size that satisfies the Armijo condition.
        int: Number of iterations taken.
    """
    alpha = method.step_size  # Initialize the step size to a pre-defined value
    tau = options.tau  # Define a reduction factor for the step size
    counter = 0

    while problem.compute_f(x + alpha * d) > (f + (c1 * alpha * np.dot(g.T, d))):
        alpha = alpha * options.tau  # Update alpha until the Armijo condition is satisfied
        counter += 1

    return alpha, counter

def Wolfe(x, f, d, g, c1, c2, problem, method, options):
    """
    Perform a Wolfe line search to find a step size that satisfies both Armijo and curvature conditions.

    Args:
        x (numpy.ndarray): Current iterate.
        f (float): Current objective function value.
        d (numpy.ndarray): Search direction.
        g (numpy.ndarray): Gradient of the objective function.
        c1 (float): Armijo condition parameter.
        c2 (float): Curvature condition parameter.
        problem (Problem): Problem object defining the optimization problem.
        method (Method): Method object defining optimization method settings.
        options (Options): Options object defining optimization options.

    Returns:
        float: The step size that satisfies both Armijo and curvature conditions.
        int: Number of iterations taken.
    """
    alpha = method.step_size  # Initialize the step size to a pre-defined value
    tau = options.tau  # Define a reduction factor for the step size
    counter = 0

    while problem.compute_f(x + alpha * d) > (problem.compute_f(x) + (c1 * alpha * np.dot(g.T, d))) \
            and np.abs(np.dot(problem.compute_g(x + alpha * d), d)) > c2*np.abs(np.dot(problem.compute_g(x), d)):
        alpha = alpha * options.tau  # Reduce the step size by multiplying it with the reduction factor
        counter += 1

    return alpha, counter

def Weak_Wolfe(x, f, d, g, c1, c2, problem, method, options):
    """
    Perform a Weak Wolfe line search to find a step size that satisfies the Armijo condition and a weak curvature condition.

    Args:
        x (numpy.ndarray): Current iterate.
        f (float): Current objective function value.
        d (numpy.ndarray): Search direction.
        g (numpy.ndarray): Gradient of the objective function.
        c1 (float): Armijo condition parameter.
        c2 (float): Curvature condition parameter.
        problem (Problem): Problem object defining the optimization problem.
        method (Method): Method object defining optimization method settings.
        options (Options): Options object defining optimization options.

    Returns:
        float: The step size that satisfies both Armijo and weak curvature conditions.
    """
    alphabar = method.step_size  # Initialize the step size to a pre-defined value
    alpha_high = 1000
    alpha_low = 0
    c = 0.5
    counter = 0
    tau = options.tau  # Define a reduction factor for the step size

    while True:
        if problem.compute_f(x + alphabar * d) <= (problem.compute_f(x) + (c1 * alphabar * np.dot(g.T, d))):
            if np.dot(problem.compute_g(x + alphabar * d), d) >= c2*np.dot(problem.compute_g(x), d):
                alpha = alphabar  # Step size set if both conditions are satisfied, and break the loop
                break

        if problem.compute_f(x + alphabar * d) <= (problem.compute_f(x) + (c1 * alphabar * np.dot(g.T, d))):
            alpha_low = alphabar
        else:
            alpha_high = alphabar

        alphabar = (c * alpha_low) + (1 - c) * alpha_high  # Update alphabar using thresholds

    return alpha


In [ ]:
def GDStep(x, f, g, problem, method, options):
    """"
    This function implements Gradient Descent for unconstrained optimization problems. It sets the descent direction to be 
    the negative gradient, gets step size using either wolfe or armijo condition and then update based on the step size found
    and returns new point x, function value and gradient at new point.

    The function returns the updated values of x, f, g
    Args:
    x (numpy.ndarray): Initial guess for the solution
    f (float): Value of the function at x
    g (numpy.ndarray): Gradient of the function at x
    problem (Problem): Object representing the optimization problem to be solved
    method (Method): Object representing the optimization method to use
    options (Options): Object containing the options for the optimization method

    Returns:
    numpy.ndarray: Solution at the next iteration
    float: Value of the function at the next iteration
    numpy.ndarray: Gradient of the function at the next iteration
    """

    d = -g  # Set the search direction d to be -gradient

    # Perform a line search using the specified step type
    if method.step_type == 'Backtracking':
        c1 = options.c_1_ls  # c1 for line search
        alpha, counter = ArmijoLineSearch(
            x, f, d, g, c1, problem, method, options)  # call armijo line search
    elif method.step_type == 'Wolfe':
        c1 = options.c_1_ls  # c1 for line search
        c2 = options.c_2_ls  # c2 for line search
        # Call wolfe conditions to get alpha
        alpha, counter = Wolfe(x, f, d, g, c1, c2, problem, method, options)
    else:
        print('Warning: step type is not defined')

    # Update the position, function value, and gradient using the step size alpha and search direction d
    x_new = x + alpha*d
    f_new = problem.compute_f(x_new)
    g_new = problem.compute_g(x_new)

    # Return the updated position, function value, and gradient
    return x_new, f_new, g_new, counter

In [7]:
def Newton(x, f, g, H, problem, method, options):
    """
    This function implements Newton method for unconstrained optimization problems.The Cholesky function is used to compute the 
    Cholesky decomposition of the Hessian matrix H. The parameter beta is used to add a regularization term to the diagonal 
    of H if necessary to ensure that H is positive definite. The Newton step direction d is computed by solving the linear
    system L@L.T@d = -g, where L is the Cholesky decomposition of H. 

The function returns the updated values of x, f, g, and H.
    Args:
    x (numpy.ndarray): Initial guess for the solution
    f (float): Value of the function at x
    g (numpy.ndarray): Gradient of the function at x
    H (np.ndarray) : Hessian of function at point x0
    problem (Problem): Object representing the optimization problem to be solved
    method (Method): Object representing the optimization method to use
    options (Options): Object containing the options for the optimization method

    Returns:
    numpy.ndarray: Solution at the next iteration
    float: Value of the function at the next iteration
    numpy.ndarray: Gradient of the function at the next iteration
    numpy.ndarray: Hessian at xnew which is x+(alpha*d)
    """

    # Define beta and the function Cholesky that computes the Cholesky decomposition of a matrix with a regularization parameter beta
    beta = options.B
  # Cholesky subroutine
    n = H.shape[0]
    # Find the minimum diagonal element of A
    diag_min = np.min(np.diag(H))
    if diag_min > 0:
        eta = 0
    else:
        eta = -1 * diag_min + options.B
    # Perform the Cholesky decomposition with the regularized matrix A + eta*I until it succeeds, or until the maximum number of iterations is reached
    for k in range(1000):
        try:
            L = np.linalg.cholesky(H + eta * np.identity(n))
            break
        except:
            eta = np.max([2 * eta, options.B])
   # Compute the search direction d as the solution of the Newton system H*d = -g using the Cholesky decomposition
    d = np.matmul(np.linalg.inv(H + eta * np.identity(n)), -1 * g)  # step size

    # Perform the line search to find the step size alpha
    if method.step_type == 'Backtracking':
        c1 = options.c_1_ls
        alpha, counter = ArmijoLineSearch(
            x, f, d, g, c1, problem, method, options)

    elif method.step_type == 'Wolfe':
        c1 = options.c_1_ls
        c2 = options.c_2_ls
        alpha, counter = Wolfe(x, f, d, g, c1, c2, problem, method, options)
    else:
        print('Warning: step type is not defined')

    # Update the variables with the new values obtained by taking a step in the search direction with the step size alpha
    x_new = x + alpha * d
    f_new = problem.compute_f(x_new)
    g_new = problem.compute_g(x_new)
    H_new = problem.compute_H(x_new)

    # Return the updated values of x, f, g, and H
    return x_new, f_new, g_new, H_new, counter

In [4]:
def BFGS(x, f, g, H_inv, problem, method, options):
    """
    This function implements the Broyden-Fletcher-Goldfarb-Shanno algorithm  for unconstrained 
    optimization problems.The algorithm builds a quasi-Newton approximation to the Hessian matrix H by updating it iteratively. 

    Args:
    x (numpy.ndarray): Initial guess for the solution
    f (float): Value of the function at x
    g (numpy.ndarray): Gradient of the function at x
    H (np.ndarray) : Hessian starts with identity and then is the update of each BFGS iteration
    problem (Problem): Object representing the optimization problem to be solved
    method (Method): Object representing the optimization method to use
    options (Options): Object containing the options for the optimization method

    Returns:
    numpy.ndarray: Solution at the next iteration
    float: Value of the function at the next iteration
    numpy.ndarray: Gradient of the function at the next iteration
    numpy.ndarray: Hessian at next iteration following BFGS update or no update
    list: Updated list of previous displacement vectors
    list: Updated list of previous gradient differences
    """

    # Set the search direction d to be -H*g, where H is the approximate inverse Hessian matrix
    # and g is the gradient vector
    d = -H_inv @ g

    if method.step_type == 'Backtracking':
        # Perform backtracking line search to find the step size alpha that satisfies the Armijo condition
        c1 = options.c_1_ls  # constant for Armijo condition
        alpha, c = ArmijoLineSearch(x, f, d, g, c1, problem, method, options)

    elif method.step_type == 'Wolfe':
        # Perform Wolfe line search to find the step size alpha that satisfies both Armijo and curvature conditions
        c1 = options.c_1_ls  # constant for Armijo condition
        c2 = options.c_2_ls  # constant for curvature condition
        alpha, c = Wolfe(x, f, d, g, c1, c2, problem, method, options)

    else:
        # If an unsupported line search method is used, print a warning message
        print('Warning: step type is not defined')

    # Update x, f, and g based on the chosen step size alpha and search direction d
    x_new = x + alpha * d
    f_new = problem.compute_f(x_new)
    g_new = problem.compute_g(x_new)

    # define s_k and y_k
    s_k = x_new - x
    y_k = g_new - g
    n = H_inv.shape[0]

    # update H inverse approximation if s_k.T @ y_k is suffficently PSD
    if s_k.T @ y_k >= options.term_tol * np.linalg.norm(s_k, ord=2) * np.linalg.norm(y_k, ord=2):
        rho_k = 1 / (y_k.T @ s_k)
        H_inv_new = (np.identity(n) - rho_k * np.outer(s_k, y_k)) @ H_inv @ (
            np.identity(n) - rho_k * np.outer(y_k, s_k)) + rho_k * np.outer(s_k, s_k)
    else:
        # print('skipped update!')
        H_inv_new = H_inv

    # Return the updated values of x, f, g, and H, as well as the change in x and g
    return x_new, f_new, g_new, H_inv_new, c

In [6]:
def DFP(x, f, g, H_inv, problem, method, options):
    """
    This function implements the DFP (Davidon-Fletcher-Powell) algorithm for unconstrained optimization problems.
    The algorithm builds a quasi-Newton approximation to the Hessian matrix H by updating it iteratively. 

    Args:
    x (numpy.ndarray): Initial guess for the solution
    f (float): Value of the function at x
    g (numpy.ndarray): Gradient of the function at x
    H (np.ndarray) : Hessian starts with identity and then is the update of each DFP iteration
    problem (Problem): Object representing the optimization problem to be solved
    method (Method): Object representing the optimization method to use
    options (Options): Object containing the options for the optimization method

    Returns:
    numpy.ndarray: Solution at the next iteration
    float: Value of the function at the next iteration
    numpy.ndarray: Gradient of the function at the next iteration
    numpy.ndarray: Hessian at next iteration following DFP update or no update
    """

  # Set the search direction d to be -g
    # Set the search direction d to be -H*g, where H is the approximate inverse Hessian matrix
    # and g is the gradient vector
    d = -H_inv @ g

    if method.step_type == 'Backtracking':
        # Perform backtracking line search to find the step size alpha that satisfies the Armijo condition
        c1 = options.c_1_ls  # constant for Armijo condition
        alpha, c = ArmijoLineSearch(x, f, d, g, c1, problem, method, options)

    elif method.step_type == 'Wolfe':
        # Perform Wolfe line search to find the step size alpha that satisfies both Armijo and curvature conditions
        c1 = options.c_1_ls  # constant for Armijo condition
        c2 = options.c_2_ls  # constant for curvature condition
        alpha, c = Wolfe(x, f, d, g, c1, c2, problem, method, options)

    else:
        # If an unsupported line search method is used, print a warning message
        print('Warning: step type is not defined')

    # Update x, f, and g based on the chosen step size alpha and search direction d
    x_new = x + alpha * d
    f_new = problem.compute_f(x_new)
    g_new = problem.compute_g(x_new)

    # define s_k and y_k
    s_k = x_new - x
    y_k = g_new - g
    n = H_inv.shape[0]

    # update H inverse approximation if s_k.T @ y_k is suffficently PSD
    if s_k.T @ y_k >= options.term_tol * np.linalg.norm(s_k, ord=2) * np.linalg.norm(y_k, ord=2):
        rho = 1 / np.dot(s_k, y_k)
        term1 = H_inv
        term2 = (H_inv@np.outer(y_k, y_k)@H_inv) / \
            (np.dot(np.matmul(y_k, H_inv), y_k))
        term3 = (np.outer(s_k, s_k)) / (np.dot(s_k, y_k))
        H_inv_new = term1 - term2 + term3
    else:  # If no sufficient increase hessian not updated.
        H_inv_new = H_inv

    # Return the updated values
    return x_new, f_new, g_new, H_inv_new, c

## Trust Region Subproblem Algorithm

In [3]:
def CG_Steihaug(x, f, g, H, Delta, problem, method, options):
    """
    The function CG_Steihaug implements the conjugate gradient method with the Steihaug stopping criterion
    for solving the quadratic subproblem in the trust-region method.

    Arguments:

    x: the current iterate (array)
    f: the value of the objective function at the current iterate (float)
    g: the gradient of the objective function at the current iterate (array)
    H: the Hessian of the objective function at the current iterate (2D array)
    Delta: the trust-region radius (float)
    problem: an object that defines the optimization problem being solved (class)
    method: an object that defines the optimization method being used (class)
    options: an object that contains options for the optimization algorithm (class)

    Returns:

    dk: the solution to the trust-region subproblem (array)
    """

    def quadratic_formula(z, p, Delta):
        p_root = (-2 * z @ p + np.sqrt((-2 * z @ p)**2 - 4 * np.linalg.norm(p)
                  ** 2 * (np.linalg.norm(z)**2 - Delta**2))) / (2 * np.linalg.norm(p)**2)

        return p_root

    z = np.zeros(len(x))  # Initialize z to 0 vector
    r = g  # Initialize r to the gradient vector
    p, Bk = -g.copy(), H  # Initialize p and Bk to -r and Hessian matrix respectively

    r_l2norm = np.linalg.norm(r, ord=2)  # Compute l2-norm of r

    if r_l2norm < options.term_tol_CG:  # Check if norm of r is smaller than tolerance
        dk = z  # If yes, set dk to 0 vector and return
        return dk

    for j in range(options.max_iterations_CG):  # Loop over maximum number of iterations
        if p.T @ Bk @ p <= 0:  # Check if p^T Bk p is non-positive
            # Find τ such that dk = zj + τpj minimizes mk (dk ) and satisfies ||dk|| = ∆k
            # Compute dk using QuadraticFormula and set to dk
            dk = z+(p*quadratic_formula(z, p, Delta))
            return dk  # Return dk

        alpha = r.T @ r / (p.T @ Bk @ p)  # Compute step size alpha

        z_next = z + alpha * p  # Compute new value of z

        # Check if norm of z_next is greater than Delta
        if np.linalg.norm(z_next, ord=2) >= Delta:
            # Find τ such that dk = zj + τpj minimizes mk (dk ) and satisfies kdkk = ∆k
            # Compute dk using QuadraticFormula and set to dk
            dk = z+(p*quadratic_formula(z, p, Delta))
            # print('greater than delta')
            return dk  # Return dk

        r_next = r + alpha * (Bk @ p)  # Compute new value of r

        # Check if norm of r_next is smaller than tolerance
        if np.linalg.norm(r_next) <= options.term_tol_CG:
            dk = z_next  # If yes, set dk to z_next and return
            return dk

        B_next = (r_next@r_next) / (r@r)  # Compute new value of B_next
        # print(B_next.shape)
        p_next = -r_next + (B_next*p)  # Compute new value of p_next

        r = r_next
        p = p_next
        z = z_next  # Update r, p and z to r_next, p_next and z_next respectively

## Trust Region Method

In [1]:
def TRNewtonCG(x, f, g, H, delta, problem, method, options):
    """
    TRNewtonCG is a function that implements the trust-region Newton-CG optimization method for minimizing a
    given objective function subject to constraints, using the CG-Steihaug method to solve the trust-region subproblem.

    Arguments:

    x: the initial guess for the optimization variables
    f: the objective function to be minimized
    g: the gradient of the objective function
    H: the Hessian matrix of the objective function
    delta: the size of the trust-region
    problem: an object that defines the problem to be solved
    method: a string indicating the optimization method to be used
    options: an object that contains options for the optimization method

    Returns:

    x_new: the new iterate of the optimization variables
    f_new: the value of the objective function at x_new
    g_new: the gradient of the objective function at x_new
    H_new: the Hessian matrix of the objective function at x_new
    delta_new: the updated size of the trust-region.
    """

    # Set the trust region c1 and c2 parameters
    c1 = options.c_1_tr
    c2 = options.c_2_tr

    # Solve the Trust Region subproblem approximately using the Conjugate Gradient method
    dk = CG_Steihaug(x, f, g, H, delta, problem, method, options)
    # print(dk)

    # Compute the numerator in the ratio
    numerator = problem.compute_f(x) - problem.compute_f(x+dk)

    # Compute the denominator in the ratio using the quadratic approximation model
    d0 = np.zeros((len(x)))
    # model = lambda d: f + g.T @ d + .5*( d.T @ H @ d)
    denominator = (f + g.T @ d0 + .5*(d0.T @ H @ d0)) - \
        (f + g.T @ dk + .5*(dk.T @ H @ dk))

    # Compute the ratio
    rho = numerator / denominator

    # Accept or reject the step based on the ratio and update the trust region radius
    if rho > c1:
        x_new = x + dk
        delta_new = delta

        f_new = problem.compute_f(x_new)
        g_new = problem.compute_g(x_new)
        H_new = problem.compute_H(x_new)

        if rho > c2:
            delta_new = 2 * delta
    else:
        x_new, f_new, g_new,  H_new = x, f, g, H
        delta_new = .5 * delta

    # Return the new point and the updated trust region radius
    return x_new, f_new, g_new, H_new, delta_new

In [ ]:
def TRSR1CG(x, f, g, H, delta, problem, method, options):
    """
    his function implements the Trust-Region Symmetric Rank-1 (TRSR1) method for solving unconstrained optimization problems.
    using the CG-Steihaug method to solve the trust-region subproblem.

     Arguments:

     x: the initial guess for the optimization variables
     f: the objective function to be minimized
     g: the gradient of the objective function
     H: Hessian is initially the identity and then updated based on Rank 1 Update.
     delta: the size of the trust-region
     problem: an object that defines the problem to be solved
     method: a string indicating the optimization method to be used
     options: an object that contains options for the optimization method

     Returns:

     x_new: the new iterate of the optimization variables
     f_new: the value of the objective function at x_new
     g_new: the gradient of the objective function at x_new
     H_new: the Hessian matrix of the objective function at x_new
     delta_new: the updated size of the trust-region.
    """

    # Initialize variables
    Bk = H
    r = options.r
    eta = 1e-3
    tol = options.term_tol_CG
    counter = 0

    x_list = []
    f_list = []
    c_list = []

    while np.linalg.norm(g) > tol and counter < 50:
        dk = CG_Steihaug(x, f, g, H, delta, problem, method, options)

        # Compute actual and predicted reduction
        yk = problem.compute_g(x + dk) - problem.compute_g(x)
        ared = problem.compute_f(x) - problem.compute_f(x + dk)
        pred = -((problem.compute_g(x).T @ dk) + 0.5 * (dk.T @ Bk @ dk))

        # Check if the step is acceptable
        if ared / pred > eta:
            x_new = x + dk
        else:
            x_new = x

        # Update trust region radius
        if ared / pred > 0.75:
            if np.linalg.norm(dk) <= 0.8 * delta:
                delta_new = delta
            else:
                delta_new = 2 * delta
        elif 0.1 <= ared / pred <= 0.75:
            delta_new = delta
        else:
            delta_new = 0.5 * delta

        # Update Bk using SR1
        term = yk - (Bk @ dk)
        if np.abs(dk.T @ term) >= r * np.linalg.norm(dk) * np.linalg.norm(term):
            Bk_new = Bk + (np.outer(term, term)) / (term.T @ dk)
        else:
            Bk_new = Bk

        # Update variables
        x = x_new
        f = problem.compute_f(x_new)
        g = problem.compute_g(x_new)
        H = problem.compute_H(x_new)
        Bk = Bk_new
        delta = delta_new
        counter += 1

        # update lists
        x_list.append(x)
        f_list.append(f)

    return x, f, x_list, f_list, c_list